Model-based collaborative filtering (CF) based on:
- SVD
- Deep learning models 

Let's go through Singular Vector Decomposition (SVD) first, a dimensionality reduction technique that is used in modern model-based CF recommender system. The other type of CF recommender system is the memory-based CF. We will also do another model-based RS based on deep learning networks
in the latter part of this notebook. 

Somehow, since I have not updated Mac to latest version, Keras installation fails. Return when I update. 
See https://github.com/khanhnamle1994/movielens/blob/master/Deep_Learning_Model.ipynb

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn import model_selection, metrics, preprocessing

# Reading ratings file
ratings = pd.read_csv('../data/ratings.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating', 'timestamp'])

# Reading users file
users = pd.read_csv('../data/users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('../data/movies.csv', sep='\t', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

In [2]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies)) 

Number of users = 6040 | Number of movies = 3706


In [4]:
ratings_1 = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0) 

In [5]:
R = ratings_1.to_numpy()
user_ratings_mean = np.mean(R, axis = 1)
user_ratings_mean

array([0.05990286, 0.12924987, 0.05369671, ..., 0.02050729, 0.1287102 ,
       0.3291959 ])

In [6]:
ratings_1_sub_mean = R - user_ratings_mean.reshape(-1, 1)

In [7]:
# Check how sparse

sparsity = round(1.0 - len(ratings) / float(n_users * n_movies), 3)
print ('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%') 

# Number of ratings: 0.045*6040*3706 \sim 1007291

The sparsity level of MovieLens1M dataset is 95.5%


In [8]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(ratings_1_sub_mean, k = 50)

In [9]:
U.shape, Vt.shape

((6040, 50), (50, 3706))

In [10]:
sigma = np.diag(sigma)
all_user_predicted_ratings = (U@sigma)@Vt + user_ratings_mean.reshape(-1, 1)


In [11]:
all_user_predicted_ratings.shape

(6040, 3706)

In [12]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = ratings_1.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.288861,0.143055,-0.195080,-0.018843,0.012232,-0.176604,-0.074120,0.141358,-0.059553,-0.195950,...,0.027807,0.001640,0.026395,-0.022024,-0.085415,0.403529,0.105579,0.031912,0.050450,0.088910
1,0.744716,0.169659,0.335418,0.000758,0.022475,1.353050,0.051426,0.071258,0.161601,1.567246,...,-0.056502,-0.013733,-0.010580,0.062576,-0.016248,0.155790,-0.418737,-0.101102,-0.054098,-0.140188
2,1.818824,0.456136,0.090978,-0.043037,-0.025694,-0.158617,-0.131778,0.098977,0.030551,0.735470,...,0.040481,-0.005301,0.012832,0.029349,0.020866,0.121532,0.076205,0.012345,0.015148,-0.109956
3,0.408057,-0.072960,0.039642,0.089363,0.041950,0.237753,-0.049426,0.009467,0.045469,-0.111370,...,0.008571,-0.005425,-0.008500,-0.003417,-0.083982,0.094512,0.057557,-0.026050,0.014841,-0.034224
4,1.574272,0.021239,-0.051300,0.246884,-0.032406,1.552281,-0.199630,-0.014920,-0.060498,0.450512,...,0.110151,0.046010,0.006934,-0.015940,-0.050080,-0.052539,0.507189,0.033830,0.125706,0.199244


In [13]:
user_row_number = 1310 - 1 # User ID starts at 1, not 0
sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
sorted_user_predictions

movie_id
1097    1.571580
1090    1.373596
1674    1.292395
1196    1.263977
1961    1.153507
          ...   
2712   -0.334191
3578   -0.362995
910    -0.368467
1304   -0.494537
1136   -0.512347
Name: 1309, Length: 3706, dtype: float64

In [14]:
user_data = ratings[ratings.user_id == (1310)]
user_data

,user_id,movie_id,rating,timestamp
215928,1310,2988,3,974781935
215929,1310,1293,5,974781839
215930,1310,1295,2,974782001
215931,1310,1299,4,974781701
215932,1310,2243,4,974782001
215933,1310,2248,5,974781573
215934,1310,2620,5,974781573
215935,1310,3683,5,974781935
215936,1310,3685,4,974781935
215937,1310,1185,4,974781839


In [15]:
user_full = user_data.merge(movies).sort_values(['rating'], ascending=False)
user_full

,user_id,movie_id,rating,timestamp,title,genres
5,1310,2248,5,974781573,Say Anything... (1989),Comedy|Drama|Romance
6,1310,2620,5,974781573,This Is My Father (1998),Drama|Romance
7,1310,3683,5,974781935,Blood Simple (1984),Drama|Film-Noir
15,1310,1704,5,974781573,Good Will Hunting (1997),Drama
1,1310,1293,5,974781839,Gandhi (1982),Drama
12,1310,3101,4,974781573,Fatal Attraction (1987),Thriller
11,1310,1343,4,974781534,Cape Fear (1991),Thriller
20,1310,2000,4,974781892,Lethal Weapon (1987),Action|Comedy|Crime|Drama
18,1310,3526,4,974781892,Parenthood (1989),Comedy|Drama
17,1310,3360,4,974781935,Hoosiers (1986),Drama


In [16]:
print ('User {0} has already rated {1} movies.'.format(1310, user_full.shape[0]))

User 1310 has already rated 24 movies.


In [17]:
sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False)
sorted_user_predictions

movie_id
1097    1.571580
1090    1.373596
1674    1.292395
1196    1.263977
1961    1.153507
          ...   
2712   -0.334191
3578   -0.362995
910    -0.368467
1304   -0.494537
1136   -0.512347
Name: 1309, Length: 3706, dtype: float64

In [18]:
num_recommendations = 10
recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index()).
         rename(columns = {user_row_number: 'Pred'}).sort_values('Pred', ascending = False).iloc[:num_recommendations, :-1]
                      )

recommendations

,movie_id,title,genres
1527,1674,Witness (1985),Drama|Romance|Thriller
1769,1961,Rain Man (1988),Drama
1115,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1144,1242,Glory (1989),Action|Drama|War
1130,1225,Amadeus (1984),Drama
1201,1302,Field of Dreams (1989),Drama
1148,1246,Dead Poets Society (1989),Drama
1770,1962,Driving Miss Daisy (1989),Drama
1766,1957,Chariots of Fire (1981),Drama
1827,2020,Dangerous Liaisons (1988),Drama|Romance


## Let's try some model evaluation now using Surprise

In [19]:
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

# Split the dataset for 5-fold evaluation
# Use the SVD algorithm.
svd = SVD()
cross_validate(svd, data, measures=['RMSE'], cv=5)
# measures=['RMSE', 'MAE']

{'test_rmse': array([0.87301567, 0.87422212, 0.87504445, 0.87383102, 0.8739564 ]),
 'fit_time': (4.545872211456299,
  4.751018047332764,
  4.798647165298462,
  4.483170986175537,
  4.245319128036499),
 'test_time': (0.9664578437805176,
  0.7485971450805664,
  0.8811099529266357,
  0.8153841495513916,
  0.7338628768920898)}

In [20]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [21]:
ratings[ratings['user_id'] == 1310] # Ratings given by User 1310

,user_id,movie_id,rating,timestamp
215928,1310,2988,3,974781935
215929,1310,1293,5,974781839
215930,1310,1295,2,974782001
215931,1310,1299,4,974781701
215932,1310,2243,4,974782001
215933,1310,2248,5,974781573
215934,1310,2620,5,974781573
215935,1310,3683,5,974781935
215936,1310,3685,4,974781935
215937,1310,1185,4,974781839


In [22]:
svd.predict(1310, 1674)

Prediction(uid=1310, iid=1674, r_ui=None, est=4.163971472542352, details={'was_impossible': False})

In [23]:
svd.predict(1310, 1961)

Prediction(uid=1310, iid=1961, r_ui=None, est=4.079132944495351, details={'was_impossible': False})

Note that MovieID 1674, 1961 were both suggested by our earlier computation. Instead of SVD we could have also
used NMF (Non negative Matrix Factorization). We will do it and see how the RMSE and compare to 0.873 from SVD. 

In [24]:
#from surprise import NMF
#algo = NMF()
#cross_validate(algo, data, measures=['RMSE'], cv=5)

RMSE of 0.915. Therefore, SVD does better! 

## Now deep learning stuff! 

In [25]:
class MovieDataset:
    def __init__(self, users, movies, ratings):
        self.users = users
        self.movies = movies
        self.ratings = ratings
    # len(movie_dataset)
    def __len__(self):
        return len(self.users)
    # movie_dataset[1] 
    def __getitem__(self, item):

        users = self.users[item] 
        movies = self.movies[item]
        ratings = self.ratings[item]
        
        return {
            "users": torch.tensor(users, dtype=torch.long),
            "movies": torch.tensor(movies, dtype=torch.long),
            "ratings": torch.tensor(ratings, dtype=torch.long),
        }

In [26]:
class RecSysModel(nn.Module):
    def __init__(self, n_users, n_movies):
        super().__init__()
        # trainable lookup matrix for shallow embedding vectors
        
        self.user_embed = nn.Embedding(n_users, 32)
        self.movie_embed = nn.Embedding(n_movies, 32)
        # user, movie embedding concat
        self.out = nn.Linear(64, 1)

    
    def forward(self, users, movies, ratings=None):
        user_embeds = self.user_embed(users)
        movie_embeds = self.movie_embed(movies)
        output = torch.cat([user_embeds, movie_embeds], dim=1)
        
        output = self.out(output)
        
        return output

In [27]:
ratings.user_id.nunique()
ratings = ratings.iloc[:5000]
ratings.describe()


,user_id,movie_id,rating,timestamp
count,5000.000000,5000.000000,5000.000000,5.000000e+03
mean,19.786000,1835.782400,3.574400,9.785097e+08
std,9.505045,1079.542711,1.081064,2.020025e+06
min,1.000000,1.000000,1.000000,9.781004e+08
25%,11.000000,1013.000000,3.000000,9.781360e+08
50%,19.500000,1777.000000,4.000000,9.781975e+08
75%,26.000000,2722.000000,4.000000,9.782730e+08
max,35.000000,3952.000000,5.000000,1.009669e+09


In [28]:
# Encode the user and movie id to start from 0 so we don't run into index out of bound with Embedding

df = ratings.copy()

lbl_user = preprocessing.LabelEncoder()
lbl_movie = preprocessing.LabelEncoder()
df.user_id = lbl_user.fit_transform(df.user_id.values)
df.movie_id = lbl_movie.fit_transform(df.movie_id.values)

df_train, df_valid = model_selection.train_test_split(df, test_size=0.1, random_state=42, stratify=df.rating.values)
train_dataset = MovieDataset(users=df_train.user_id.values,movies=df_train.movie_id.values,ratings=df_train.rating.values)
valid_dataset = MovieDataset(users=df_valid.user_id.values,movies=df_valid.movie_id.values,ratings=df_valid.rating.values)

In [29]:
import torch
from torch.utils.data import Dataset, DataLoader
train_loader = DataLoader(dataset=train_dataset,batch_size=4,shuffle=True,num_workers=2) 
validation_loader = DataLoader(dataset=valid_dataset,batch_size=4,shuffle=True,num_workers=2) 

dataiter = iter(train_loader)
dataloader_data = dataiter.next() 
#print(dataloader_data)

AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/apple/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/apple/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'MovieDataset' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/apple/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/apple/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'MovieDataset' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Users/apple/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    retu

: 

In [ ]:
model = RecSysModel(
    n_users=len(lbl_user.classes_),
    n_movies=len(lbl_movie.classes_),
)

optimizer = torch.optim.Adam(model.parameters())  
sch = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.7)

loss_func = nn.MSELoss()

In [ ]:
print(len(lbl_user.classes_))
print(len(lbl_movie.classes_))
print(ratings.movie_id.max())
print(len(train_dataset))

6040
3706
3952
900188


In [ ]:
user_embed = nn.Embedding(len(lbl_user.classes_), 32)
movie_embed = nn.Embedding(len(lbl_movie.classes_), 32)

out = nn.Linear(64, 1)

In [ ]:
epochs = 1
total_loss = 0
plot_steps, print_steps = 5000, 5000
step_cnt = 0
all_losses_list = [] 

model.train() 
for epoch_i in range(epochs):
    for i, train_data in enumerate(train_loader):
        output = model(train_data["users"], 
                       train_data["movies"]
                      ) 
        
        # .view(4, -1) is to reshape the rating to match the shape of model output which is 4x1
        rating = train_data["ratings"].view(4, -1).to(torch.float32)

        loss = loss_func(output, rating)
        total_loss = total_loss + loss.sum().item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        step_cnt = step_cnt + len(train_data["users"])
        

        if(step_cnt % plot_steps == 0):
            avg_loss = total_loss/(len(train_data["users"]) * plot_steps)
            print(f"epoch {epoch_i} loss at step: {step_cnt} is {avg_loss}")
            all_losses_list.append(avg_loss)
            total_loss = 0 # reset total_loss

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/apple/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/apple/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'MovieDataset' on <module '__main__' (built-in)>


KeyboardInterrupt: 